In [1]:
import scanpy as sc
import squidpy as sq
import spatialdata as sd
import spatialdata_io as si
import numpy as np
import pandas as pd
import anndata
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import seaborn as sns
import spatialdata_plot
from spatialdata.transformations import (
    Affine,
    Identity,
    MapAxis,
    Scale,
    Sequence,
    Translation,
    get_transformation,
    get_transformation_between_coordinate_systems,
    set_transformation,
)
from spatialdata.models import ShapesModel
from spatialdata.transformations import (
    align_elements_using_landmarks,
    get_transformation_between_landmarks,
)
from napari_spatialdata import Interactive


❗ You are running 3.11.3
Only python versions 3.7~3.10 are currently tested, use at your own risk.


# Load Data

In [2]:
datasets = ['LIB5472833_SAM24434294','LIB5472834_SAM24434295', 'LIB5472835_SAM24434296',
            'LIB5472836_SAM24434297','LIB5472837_SAM24434298','LIB5472838_SAM24434299']
data_list=[]
for dataset in datasets:
    data_list.append(sd.read_zarr(f'/Users/lucast3/Documents/VisiumAnalysis/data/{dataset}_updated.zarr'))

# Add HE slide

In [3]:
import cv2
x=0
# loop through each dataset adding Abeta image, metadata, and process spots using standard pipeline
for dataset in datasets:
    
    #read in Hi res HE image
    im = cv2.imread(f'/Users/lucast3/Documents/VisiumAnalysis/HE/{dataset.split("_")[0]}_40x.tif')
    
    #Parse
    img_sd = sd.models.Image2DModel.parse(im,
                                  dims=("y","x","c"))
    #Add to object
    data_list[x].images["HE_fullres"]= img_sd
    
    #Transform
    scale = Scale([0.6666667,0.6666667], axes=("y","x"))
    set_transformation(data_list[x].images["HE_fullres"],scale,to_coordinate_system="downscaled_hires")
    x=x+1

INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           
INFO     Transposing `data` of type: <class 'dask.array.core.Array'> to ('c', 'y', 'x').                           


# Mark landmarks for each dataset

## LIB5472833

In [12]:
Interactive(data_list[0])

## LIB5472834


In [15]:
Interactive(data_list[1])

## LIB5472835


In [6]:
Interactive(data_list[2])

## LIB5472836


In [7]:
Interactive(data_list[3])

## LIB5472837


In [8]:
Interactive(data_list[4])

## LIB5472838


In [24]:
Interactive(data_list[5])

# Align images

In [21]:
x=0
# loop through each dataset aligning images
for dataset in datasets:
    affine = get_transformation_between_landmarks(
    references_coords=data_list[x]["HELandmarks"], moving_coords=data_list[x]["AlignedLandmarks"]
    )
    affine1 = align_elements_using_landmarks(
        moving_coords=data_list[x]["AlignedLandmarks"],
        references_coords=data_list[x]["HELandmarks"],
        moving_element=data_list[x][f'{dataset}_hires_image'],
        reference_element=data_list[x]["HE_fullres"],
        reference_coordinate_system="downscaled_hires",
        moving_coordinate_system="aligned",
        new_coordinate_system="FinalAlignment",
    )
    sequence=Sequence([get_transformation(data_list[x].shapes[f'{dataset}'],get_all=True)["aligned"],affine])
    set_transformation(data_list[x].shapes[f'{dataset}'],sequence,to_coordinate_system="FinalAlignment")
    
    sequence=Sequence([get_transformation(data_list[x].images["AB"],get_all=True)['aligned'], affine])
    set_transformation(data_list[x].images["AB"],sequence,to_coordinate_system="FinalAlignment")
    
    sequence=Sequence([get_transformation(data_list[x].shapes["Abeta"],get_all=True)['aligned'], affine])
    set_transformation(data_list[x].shapes["Abeta"],sequence,to_coordinate_system="FinalAlignment")


    sequence=Sequence([get_transformation(data_list[x].shapes["Cortex"],get_all=True)['aligned'], affine])
    set_transformation(data_list[x].shapes["Cortex"],sequence,to_coordinate_system="FinalAlignment")

    sequence=Sequence([get_transformation(data_list[x].shapes["Hippocampus"],get_all=True)['aligned'], affine])
    set_transformation(data_list[x].shapes["Hippocampus"],sequence,to_coordinate_system="FinalAlignment")
    
    data_list[x].write(f'/Users/lucast3/Documents/VisiumAnalysis/data/{dataset}_Final.zarr')
    x=x+1


INFO     The Zarr file used for backing will now change from                                                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472834_SAM24434295_updated.zarr to                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472834_SAM24434295_Final.zarr                            
INFO     The Zarr file used for backing will now change from                                                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472835_SAM24434296_updated.zarr to                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472835_SAM24434296_Final.zarr                            
INFO     The Zarr file used for backing will now change from                                                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472836_SAM24434297_updated.zarr to                       
         /Users/lucast3/Documents/VisiumAnalysis/data/LIB5472836_SAM2443